In [ ]:
#P4
import numpy as np
import pandas as pd
import csv
from sklearn.feature_selection import mutual_info_classif
from collections import Counter
from pprint import pprint

def id3(df,target_attribute,attribute_names,default_class=None):
    cnt=Counter(x for x in df[target_attribute])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        gains=mutual_info_classif(df[attribute_names],df[target_attribute],discrete_features=True)
        index_of_max=gains.tolist().index(max(gains))
        best_attr=attribute_names[index_of_max]
        remain=[i for i in attribute_names if i!=best_attr]
        tree={best_attr:{}}
        
        for attrval,datasubset in df.groupby(best_attr):
            subtree=id3(datasubset,target_attribute,remain,default_class)
            tree[best_attr][attrval]=subtree
        return tree

df=pd.read_csv("traintennis.csv")
attribute_names=df.columns.tolist()
attribute_names.remove("PlayTennis")
target_attribute="PlayTennis"

print(df)
for colname in df.select_dtypes("object"):
    df[colname],_=df[colname].factorize()
print(df)
tree=id3(df,target_attribute,attribute_names)
pprint(tree)

In [ ]:
#P3
import csv
with open("trainingexamples.csv") as f:
    csvfile=csv.reader(f)
    data=list(csvfile)
    specific=data[0][:-1]
    general=[["?" for i in range(len(specific))]for i in range(len(specific))]

    step=1
    for i in data:
        if i[-1]=="Y":
            for j in range(len(specific)):
                if specific[j]!=i[j]:
                    specific[j]="?"
                    general[j][j]="?"
        elif i[-1]=="N":
            for j in range(len(specific)):
                if specific[j]!=i[j]:
                    general[j][j]=specific[j]
                else:
                    general[j][j]="?"
        print("step {} of algo".format(step))
        step+=1
        print(specific)
        print(general)
    gh=[]
    for i in general:
        for j in i:
            if j!="?":
                gh.append(i)
    
    print("final specific:")
    print(specific)
    print("final general:")
    print(gh)

In [ ]:
#P9
import matplotlib.pyplot as plt
import math
import numpy as np

def local_regression(x0,X,Y,tau):
    x0=[1,x0]
    X=[[1,i] for i in X]
    X=np.asarray(X)
    w=np.exp(-1*np.sum((X-x0)**2,axis=1)/(2*tau**2))
    xw=(X.T)*w
    beta=np.linalg.pinv(xw@X)@(xw@Y)
    return (beta@x0)
def draw(tau):
    predictions=[local_regression(x0,X,Y,tau) for x0 in X]
    plt.plot(X,Y,'o',color="black")
    plt.plot(domain,predictions,color="red")
    plt.show()

X=np.linspace(-3,3,num=1000)
domain=X
Y=np.log(np.abs((X**2)-1)+0.5)
draw(10)
draw(0.1)
draw(0.001)

In [ ]:
#P6

import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data=pd.read_csv("diabetes2.csv")
print("FIRST FIVE ROWS OF DATA")
print(data.head())

X=data.iloc[:,:-1]
Y=data.iloc[:,-1]

X_TRAIN,X_TEST,Y_TRAIN,Y_TEST=train_test_split(X,Y)
#print(X_TRAIN)
#print(Y_TRAIN)
#print(X_TEST)
#print(Y_TEST)

classifier=GaussianNB()
classifier.fit(X_TRAIN,Y_TRAIN)
pred=classifier.predict(X_TEST)

print("ACTUAL")
print(Y_TEST.values)
print("PREDICTIONS")
print(pred)
print("Accuracy of NB:{}".format(accuracy_score(pred,Y_TEST)))

In [ ]:
#P8
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

data=load_iris()

X_TRAIN,X_TEST,Y_TRAIN,Y_TEST=train_test_split(data["data"],data["target"])
print(X_TEST)
print(Y_TEST)

kn=KNeighborsClassifier(n_neighbors=5)
kn.fit(X_TRAIN,Y_TRAIN)

for i in range(len(X_TEST)):
    X=X_TEST[i]
    X_NEW=np.array([X])
    prediction=kn.predict(X_NEW)
    print("ACTUAL :{}, PREDICTED: {}".format(Y_TEST[i],prediction))

print("ACCURACY OF KNN:",kn.score(X_TEST,Y_TEST))

In [ ]:
#P1
graph={
    'A':[('B',6),('F',3)],
    'B':[('C',3),('D',2)],
    'C':[('D',1),('E',5)],
    'D':[('C',1),('E',8)],
    'E':[('I',5),('J',5)],
    'F':[('G',1),('H',7)],
    'G':[('I',3)],
    'H':[('I',2)],
    'I':[('E',5),('J',3)]
}
def h(n):
    H={
        'A':10,
       'B':8,
       'C':5,
       'D':7,
       'E':3,
       'F':6,
       'G':5,
       'H':3,
       'I':1,
       'J':0
       
       }
    return H[n]
def get_neighbors(v):
    if v in graph:
        return graph[v]
    else:
        return None
def astar(start,stop):
    openset=set(start)
    closedset=set()
    g={}
    parents={}
    g[start]=0
    parents[start]=start

    while len(openset)>0:
        n=None
        for v in openset:
            if n==None or g[n]+h(n)>g[v]+h(v):
                n=v
        if n==stop or graph[n]==None:
            pass
        else:
            for (m,w) in get_neighbors(n):
                if m not in closedset and m not in openset:
                    openset.add(m)
                    parents[m]=n
                    g[m]=g[n]+w
                else:
                    if g[m]>g[n]+w:
                        g[m]=g[n]+w
                        parents[m]=n
                        if m in closedset:
                            closedset.remove(m)
                            openset.add(m)
        if n==None:
            print("path doesnt exist")
            return None
        if n==stop:
            path=[]
            while (parents[n]!=n):
                path.append(n)
                n=parents[n]
            path.append(start)        
            path.reverse()

            print("path found:{}".format(path))
            return path
        openset.remove(n)
        closedset.add(n)
    print("path doesnt exist")
    return None

astar('A','J')

In [ ]:
#P2
class Graph:
    def __init__(self,start,graph,H):
        self.start=start
        self.graph=graph
        self.H=H
        self.status={}
        self.parents={}
        self.solngraph={}
    def applyaostar(self):
        self.aostar(self.start,False)
    def getneighbors(self,v):
        return self.graph.get(v,'')
    def getheuristic(self,v):
        return self.H.get(v,0)
    def setheuristic(self,v,val):
        self.H[v]=val
    def getstatus(self,v):
        return self.status.get(v,0)
    def setstatus(self,v,val):
        self.status[v]=val
    def printsoln(self):
        print("FOR TRAVERSAL START FROM START NODE, ",self.start)
        print(self.solngraph)
        print("----------------------------------------------------------------")
    def computemincostchildnode(self,v):
        mc=0
        childnode={}
        childnode[mc]=[]
        flag=True
        for neighbors in self.getneighbors(v):
            cost=0
            nodelist=[]
            for (m,w) in neighbors:
                cost=cost+self.getheuristic(m)+w
                nodelist.append(m)
            if flag==True:
                mc=cost
                childnode[mc]=nodelist
                flag=False
            else:
                if mc>cost:
                    mc=cost
                    childnode[mc]=nodelist
        return mc,childnode[mc]
    def aostar(self,v,backtrack):
        print("HEURISTIC VALUES:",self.H)
        print("CURRENT NODE:",v)
        print("SOLN GRAPH:",self.solngraph)
        print("-----------------------------------------------------------------")
        if self.getstatus(v)>=0:
            mc,childnode=self.computemincostchildnode(v)
            self.setstatus(v,len(childnode))
            self.setheuristic(v,mc)
            solved=True

            for child in childnode:
                self.parents[child]=v
                if self.getstatus(child)!=-1:
                    solved=solved&False
            if solved==True:
                self.setstatus(v,-1)
                self.solngraph[v]=childnode
            if v!=self.start:
                self.aostar(self.parents[v],True)
            if backtrack==False:
                for child in childnode:
                    self.setstatus(child,0)
                    self.aostar(child,False)
                    
graph={
    'A':[[('B',1),('C',1)],[('D',1)]],
    'B':[[('G',1)],[('H',1)]],
    'D':[[('E',1),('F',1)]]
}
H={'A':1,'B':6,'C':12,'D':10,'E':4,'F':4,'G':5,'H':7}
g2=Graph('A',graph,H)
g2.applyaostar()
g2.printsoln()

    


In [45]:
#P5
import numpy as np
X=np.array(([2,9],[1,3],[5,6]),dtype=float) 
Y=np.array(([92],[86],[89]),dtype=float)
X=X/(np.amax(X,axis=0))
Y=Y/100

def sigmoid(x):
    return 1/(1+np.exp(-x))
def derivative_sigmoid(x):
    return x*(1-x)

epoch=5000
lr=0.1
wh=np.random.uniform(size=(2,3))
wout=np.random.uniform(size=(3,1))
bh=np.random.uniform(size=(1,3))
bout=np.random.uniform(size=(1,1))

for i in range(epoch):
    hinp=np.dot(X,wh)+bh
    hlayeract=sigmoid(hinp)
    outinp=np.dot(hlayeract,wout)+bout
    output=sigmoid(outinp)

    hgrad=derivative_sigmoid(hlayeract)
    outgrad=derivative_sigmoid(output)

    EO =Y-output
    doutput=EO*outgrad

    EH=np.dot(doutput,wout.T)
    dhidden=EH*hgrad

    wout+=np.dot(hlayeract.T,doutput)*lr
    wh+=np.dot(X.T,dhidden)*lr

print("INPUT:\n",str(X))
print("OUTPUT:\n",str(Y))
print("PREDICTED:\n",output)


INPUT:
 [[0.4        1.        ]
 [0.2        0.33333333]
 [1.         0.66666667]]
OUTPUT:
 [[0.92]
 [0.86]
 [0.89]]
PREDICTED:
 [[0.89840488]
 [0.87592566]
 [0.89576338]]


In [ ]:
#P7
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

iris=load_iris()

X=pd.DataFrame(iris.data)
X.columns=["Sepal_Length","Sepal_Width","Petal_Length","Petal_Width"]
Y=pd.DataFrame(iris.target)
Y.columns=["Targets"]

print(X)
print(Y)
colormap=np.array(["red","lime","black"])

plt.subplot(1,2,1)
plt.scatter(X.Petal_Length,X.Petal_Width,c=colormap[Y.Targets],s=40)
plt.title("REAL CLUSTERING")
plt.show()


model1=KMeans(n_clusters=3)
model1.fit(X)

plt.subplot(1,2,1)
plt.scatter(X.Petal_Length,X.Petal_Width,c=colormap[model1.labels_],s=40)
plt.title("KMEANS CLUSTERING")
plt.show()

model2=GaussianMixture(n_components=3)
model2.fit(X)

plt.subplot(1,2,1)
plt.scatter(X.Petal_Length,X.Petal_Width,c=colormap[model2.predict(X)],s=40)
plt.title("EM CLUSTERING")
plt.show()

print("ACTUAL\n",iris.target)
print("KMEANS:\n",model1.labels_)
print("EM:\n",model2.predict(X))
print("ACCURACY OF KMEANS IS {}".format(accuracy_score(Y,model1.labels_)))
print("ACCURACY OF EM IS {}".format(accuracy_score(Y,model2.predict(X))))

